In [1]:
from transformers import pipeline, AutoModelWithLMHead, AutoTokenizer
import spacy
import pandas as pd
from nltk import sent_tokenize
import torch
import dill

from transformers import DistilBertForQuestionAnswering, DistilBertTokenizerFast
from torch.utils.data import DataLoader
from transformers import AdamW
from transformers import BertTokenizer, BertForQuestionAnswering

In [2]:
torch.cuda.is_available()

True

for sentence in sen
doc = nlp(sentences[i])
for ent in doc.ents:
    if entities1.get(ent) in ["PERSON"]:
        if nltk.pos_tag([ent])[0][1] in ["NNS", "NNPS"]:
            questions.append("Who are {}?".format(ent))
        else:
            questions.append("Who is {}?".format(ent))
    elif entities1.get(ent) in ["GPE", "LOC", "FAC"]:
            questions.append("What is in {}?".format(ent))
    elif entities1.get(ent) in ["DATE"]:
        questions.append("From the reading, what happened in {}?".format(ent))
    elif entities1.get(ent) == "EVENT":
        questions.append("What happened at {}?".format(ent))
        questions.append("When did {} happen?".format(ent))
    elif entities1.get(ent) == "ORDINAL":
         questions.append("According to the passage, what happened for the {} time?".format(ent))
return questions, sentences, entities1, entities2, entities3

f = open("a1.txt", encoding = "utf-8", mode = "r")
txt = f.read()

sentences = sent_tokenize(txt)

In [8]:
import dill as pickle

with open("sq_questions", "rb") as dill_file:
    questions = pickle.load(dill_file)
    
with open("sq_contexts", "rb") as dill_file:
    contexts = pickle.load(dill_file)
    
with open("sq_answers", "rb") as dill_file:
    answers = pickle.load(dill_file)

In [9]:
len(contexts), len(questions), len(answers)

(11200, 11200, 11200)

In [10]:
train_contexts, train_questions, train_answers = contexts[:10000], questions[:10000], answers[:10000]
val_contexts, val_questions, val_answers = contexts[10000:], questions[10000:], answers[10000:]

In [11]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)
        answer['answer_end'] = end_idx

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [12]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased') #BertTokenizer.from_pretrained('bert-base-cased')

In [13]:
#DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [15]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        if answers[i]['answer_start'] == -1:
            ans = 0
        else:
            ans = answers[i]['answer_start']
        start_positions.append(encodings.char_to_token(i, ans))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)


In [16]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [18]:
#DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")
model = DistilBertForQuestionAnswering.from_pretrained("./finetuned_qa_model_combined/")

In [19]:
from tqdm import tqdm

In [20]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Device: ",device)
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    print("Epoch: ", epoch + 1)
    for batch in tqdm(train_loader):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()

Device:  cuda


  0%|                                                                                         | 0/5000 [00:00<?, ?it/s]

Epoch:  1


  0%|                                                                                         | 0/5000 [00:01<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 2.00 GiB total capacity; 426.79 MiB already allocated; 2.81 MiB free; 452.00 MiB reserved in total by PyTorch)

In [ ]:
#f = open("a1.txt", encoding = "utf-8", mode = "r")
text = 'However, Nile flood control was still the subject of very large works, including especially the canal to Lake Moeris around 2300 BC, which was likely also the source of water to the Giza pyramid complex centuries earlier.'

question = "What was  the subject of very large works, including especially the canal to Lake Moeris around 2300 BC, which was likely also the source of water to the Giza pyramid complex centuries earlier ?"

In [ ]:
inputs = tokenizer(question, text, return_tensors='pt')
start_positions = torch.tensor([1])
end_positions = torch.tensor([3])

outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
loss = outputs.loss
start_scores = outputs.start_logits
end_scores = outputs.end_logits

In [ ]:
answer_start = torch.argmax(start_scores) 
answer_end = torch.argmax(end_scores) + 1

tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

In [ ]:
model.save_pretrained('./finetuned_qa_model/') #('./finetuned_qa_model/')

In [ ]:
loaded_model = DistilBertForQuestionAnswering.from_pretrained('./finetuned_qa_model/')

In [ ]:
inputs = tokenizer(question, text, return_tensors='pt')
start_positions = torch.tensor([1])
end_positions = torch.tensor([3])#

outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
loss = outputs.loss
start_scores = outputs.start_logits
end_scores = outputs.end_logits

In [ ]:
answer_start = torch.argmax(start_scores) 
answer_end = torch.argmax(end_scores) + 1

tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

In [ ]:
questions[0], contexts[0]